# 設置環境並載入套件

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/CorpusforDDL_compute/corpus_compute


In [2]:
import pandas as pd
import re
import unicodedata
import pickle
from collections import Counter

In [3]:
# 載入事先寫好的 functions
from compute_parameters import *

# 讀入語料

In [26]:
naer_bilingual_path = '../CorpusforDDL/naer/naer_bilingual'
all_files = os.listdir(naer_bilingual_path)

In [27]:
import xml.etree.ElementTree as ET
tree = ET.parse(f'{naer_bilingual_path}/{all_files[0]}')
root = tree.getroot()

In [30]:
naer_bilingual_corpus = [x.text for x in root.iter('chsentence')]
naer_bilingual_corpus = [x for x in naer_bilingual_corpus if x is not None]

In [39]:
naer_bilingual_preprocessed = []

for text in naer_bilingual_corpus:
  joined = ''.join(text)
  split_1 = re.split(r'(?<=。」|！」|？」)', joined) # 先用 。」 ！」 ？」 分隔
  split_2 = [re.split(r'(?<=[。！？])(?!」)', x) for x in split_1] # 再用 。！？ 分隔
  split = [item for sublist in split_2 for item in sublist] # 將 list of list 攤平

  naer_bilingual_preprocessed.append(split)

### 語料格式準備 - 1
斷詞 + pos tag

In [43]:
naer_bilingual_ws_pos = [x[0] for x in naer_bilingual_preprocessed]
naer_bilingual_ws_pos[0]

'一/Neu 場場/Nf 台灣/Nc 舞蹈家/Na 與/Caa 日本/Nc 、/PUNC 泰國/Nc 、/PUNC 德國/Nc 的/DE 合作/VH 展演/VC ，/PUNC 所/D 表現/VC 的/DE 除了/P 是/SHI 舞蹈家/Na 本身/Nh 的/DE 創作/Na 能量/Na ，/PUNC 也/D 是/SHI 跨/VC 文化/Na 舞團/Na 間/Ng 磨合/Na 、/PUNC 包容/Na 與/Caa 融合/VC 的/DE 成果/Na 。'

### 語料格式準備 - 2
斷好詞，無標點符號

In [48]:
def naer_get_sentence(sentence):

  sentence_words = re.sub(r'/[A-z]+', '', sentence)
  sentence_words = re.sub(r'[^\w\s]', '', sentence_words).strip().split(' ')
  sentence_words = list(filter(None, sentence_words))

  return sentence_words

In [49]:
naer_bilingual_ws = [naer_get_sentence(sentence) for sentence in naer_bilingual_ws_pos]
naer_bilingual_ws[0]

['一',
 '場場',
 '台灣',
 '舞蹈家',
 '與',
 '日本',
 '泰國',
 '德國',
 '的',
 '合作',
 '展演',
 '所',
 '表現',
 '的',
 '除了',
 '是',
 '舞蹈家',
 '本身',
 '的',
 '創作',
 '能量',
 '也',
 '是',
 '跨',
 '文化',
 '舞團',
 '間',
 '磨合',
 '包容',
 '與',
 '融合',
 '的',
 '成果']

### 語料格式準備 - 3
完整句子，含標點符號

In [51]:
naer_bilingual_text = []
for sent in naer_bilingual_ws_pos:
  res = re.sub(r'/[A-z]+', '', sent)
  res = res.replace(' ', '')
  naer_bilingual_text.append(res)

naer_bilingual_text[0]

'一場場台灣舞蹈家與日本、泰國、德國的合作展演，所表現的除了是舞蹈家本身的創作能量，也是跨文化舞團間磨合、包容與融合的成果。'

# 句子長度

In [52]:
sentence_length = [len(sentence) for sentence in naer_bilingual_ws]
sentence_length[:10]

[33, 37, 20, 7, 19, 32, 18, 26, 13, 28]

# 高頻詞與低頻詞

In [53]:
high_low_freq = [get_high_low_freq(sentence) for sentence in naer_bilingual_ws]
high_low_freq[:10]

[{'High': 33},
 {'High': 37},
 {'High': 20},
 {'High': 7},
 {'Low': 1, 'High': 18},
 {'High': 31, 'Low': 1},
 {'High': 16, 'Low': 2},
 {'High': 26},
 {'High': 12, 'Low': 1},
 {'High': 26, 'Low': 2}]

# 詞頻

In [54]:
word_freq = [get_word_freq(sentence) for sentence in naer_bilingual_ws]
word_freq[0]

'一(3701681) 場場(202) 台灣(130718) 舞蹈家(364) 與(968432) 日本(105587) 泰國(2285) 德國(22936) 的(16029838) 合作(35908) 展演(799) 所(527327) 表現(67272) 的(16029838) 除了(75964) 是(4642407) 舞蹈家(364) 本身(44138) 的(16029838) 創作(30713) 能量(30785) 也(1590760) 是(4642407) 跨(8128) 文化(115697) 舞團(628) 間(115638) 磨合(335) 包容(4035) 與(968432) 融合(6722) 的(16029838) 成果(16253)'

# 詞彙等級

In [55]:
word_level = [get_word_level(sentence) for sentence in naer_bilingual_ws]
word_level[0]

{'第1級': 10,
 'Unknown': 7,
 '第4*級': 3,
 '第5級': 6,
 '第2級': 3,
 '第3*級': 2,
 '第4級': 1,
 '第6級': 1}

# 詞彙長度

In [56]:
long_word_count = [get_long_word_count(sentence) for sentence in naer_bilingual_ws]
long_word_count[:10]

[2, 1, 1, 0, 5, 1, 2, 1, 1, 4]

# 完整的句子

In [57]:
is_complete_sentence = [get_complete_sentence(sentence) for sentence in naer_bilingual_ws_pos]
is_complete_sentence[:10]

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']

# 完整的語境

In [58]:
is_complete_context = [get_complete_context(x) for x in naer_bilingual_ws_pos]
is_complete_context[:10]

['Y', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N']

# Black list

In [59]:
is_blacklist = [get_blacklist(x) for x in naer_bilingual_text]
is_blacklist[:10]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

# Grey list

In [60]:
is_greylist = [get_greylist(x) for x in naer_bilingual_ws_pos]
is_greylist[:10] 

['N', 'N', 'N', 'N', 'Y', 'Y', 'N', 'N', 'N', 'N']

# Make dataframe

In [61]:
naer_bilingual_df = pd.DataFrame({'sentence': naer_bilingual_text,
                        'sentence_preprocessed': [' '.join(sent) for sent in naer_bilingual_ws],
                        'sentence_length': sentence_length,
                        'word_freq': word_freq,
                        'high_low_freq': high_low_freq,
                        'word_level': word_level,
                        'long_word_count': long_word_count,
                        'is_complete_sentence': is_complete_sentence,
                        'is_complete_context': is_complete_context,
                        'is_blacklist': is_blacklist,
                        'is_greylist': is_greylist})

In [62]:
naer_bilingual_df = naer_bilingual_df[naer_bilingual_df['sentence_length']>0]
naer_bilingual_df

,sentence,sentence_preprocessed,sentence_length,word_freq,high_low_freq,word_level,long_word_count,is_complete_sentence,is_complete_context,is_blacklist,is_greylist
0,一場場台灣舞蹈家與日本、泰國、德國的合作展演，所表現的除了是舞蹈家本身的創作能量，也是跨文化...,一 場場 台灣 舞蹈家 與 日本 泰國 德國 的 合作 展演 所 表現 的 除了 是 舞蹈家...,33,一(3701681) 場場(202) 台灣(130718) 舞蹈家(364) 與(96843...,{'High': 33},"{'第1級': 10, 'Unknown': 7, '第4*級': 3, '第5級': 6,...",2,Y,Y,N,N
1,本期《光華》帶您深入跨國舞團彼此合作的方方面面，由「編舞者」跟「舞者」之間的角色切換，剖析每...,本 期 光華 帶 您 深入 跨國 舞團 彼此 合作 的 方方面面 由 編 舞 者 跟 舞者 ...,37,本(197156) 期(14058) 光華(600) 帶(105786) 您(78094) ...,{'High': 37},"{'第1級': 8, 'Unknown': 8, '第5級': 4, '第3*級': 2, ...",1,Y,Y,N,N
2,藝術家的工作，是能夠穿透對立，圓融契合，又能在矛盾的張力中，體現生命的價值。,藝術家 的 工作 是 能夠 穿透 對立 圓融 契合 又 能 在 矛盾 的 張力 中 體現 生...,20,藝術家(26163) 的(16029838) 工作(220164) 是(4642407) 能...,{'High': 20},"{'Unknown': 3, '第1級': 5, '第6級': 3, '第5級': 2, '...",1,Y,N,N,N
3,當心靈飽滿了，創作怎會枯竭？,當 心靈 飽滿 了 創作 怎會 枯竭,7,當(294625) 心靈(37927) 飽滿(2112) 了(3199291) 創作(307...,{'High': 7},"{'Unknown': 3, '第6級': 2, '第5級': 1, '第7級': 1}",0,Y,N,N,N
4,李光裕的《鬥牛系列》作品，饒富東方禪意及民俗元素，在2017年第屆威尼斯雙年展中，大放異彩。,李光裕 的 鬥牛 系列 作品 饒富 東方 禪意 及 民俗 元素 在 2017年 第 屆 威尼...,19,李光裕(Unknown) 的(16029838) 鬥牛(415) 系列(19048) 作品(...,"{'Low': 1, 'High': 18}","{'Unknown': 10, '第1級': 1, '第6級': 3, '第7級': 1, ...",5,Y,Y,N,Y
...,...,...,...,...,...,...,...,...,...,...,...
506,吳庭安說。,吳庭安 說,2,吳庭安(Unknown) 說(1285247),"{'Low': 1, 'High': 1}","{'Unknown': 1, '第1級': 1}",1,Y,N,N,Y
507,2018年，還有2更多的W春池計畫在推動中，光想像著未來無限的可能性，就讓人興奮不已。,2018年 還 有2 更多 的 W春池 計畫 在 推動 中 光 想像 著 未來 無限 的 可...,22,2018年(40) 還(625086) 有2(Unknown) 更多(44083) 的(16...,"{'Low': 3, 'High': 19}","{'Unknown': 8, '第1級': 4, '第2級': 2, '第1*級': 1, ...",3,Y,N,N,N
508,但是在一切酷翻了、炫斃的合作案背後，隱藏著春池「把自己本業做到最好的精神」，是春池玻璃的網頁...,但是 在 一切 酷 翻 了 炫斃 的 合作案 背後 隱藏 著 春池 把 自己 本業 做到 最...,44,但是(247988) 在(3284238) 一切(95523) 酷(3249) 翻(9792...,"{'High': 40, 'Low': 4}","{'第2級': 3, '第1*級': 4, '第3*級': 3, '第6級': 1, '第4...",1,Y,Y,N,N
509,2016年蔡英文總統到訪春池玻璃，走了一圈，她也有感地說：「簡單的『循環經濟』4個字，有2人...,2016年 蔡 英文 總統 到 訪春池 玻璃 走 了 一 圈 她 也 有感 地 說 簡單 的...,29,2016年(602) 蔡(2066) 英文(15308) 總統(23662) 到(54690...,"{'High': 27, 'Low': 2}","{'Unknown': 9, '第1級': 13, '第4級': 3, '第2級': 1, ...",2,Y,N,N,Y


In [63]:
naer_bilingual_df.to_csv('../results/naer_bilingual/naer_bilingual_parameters_example.csv', index = False)

## Concordance

In [64]:
from nltk.text import Text

In [65]:
target_words = ['難得', '畢竟', '的確', '難免', '總是', '有助於']
corpus = [item for sublist in naer_bilingual_ws for item in sublist]
text = Text(corpus)
dfs = []

for word in target_words:

  con_list = text.concordance_list(word)
  right_word = [x.right[0] for x in con_list]
  left_word = [x.left[-1] for x in con_list]
  context = [x.left + [word] + x.right for x in con_list]
  context = [' '.join(x) for x in context]

  df = pd.DataFrame({'left_word': left_word,
                     'target_word': word,
                     'right_word': right_word,
                     'context': context})
  dfs.append(df)

In [66]:
concordance_df = pd.concat(dfs)
concordance_df.to_csv('../results/naer_bilingual/naer_bilingual_concordance_df.csv', index = False)